In [1]:
import sys
import os
sys.path.append(os.getcwd()+'/fivepoint')
import build.fivep as f
import time
import torch
import torch.nn.functional as F
from spherical_distortion.functional import create_tangent_images, unresample
from spherical_distortion.util import *
import matplotlib.pyplot as plt
from skimage import io
import os
import cv2

import sys
import pandas as pd
import numpy as np
import _spherical_distortion_ext._mesh as _mesh
import argparse

from random import sample
import imageio
from scipy.spatial.transform import Rotation as Rot

from utils.coord    import coord_3d
from utils.ransac   import *
from utils.keypoint import *
from utils.metrics  import *
from utils.camera_recovering import *

from os import listdir
from os.path import isfile, join, isdir
from tqdm import tqdm

sys.path.append(os.getcwd()+'/SPHORB-master')

import build1.sphorb_cpp as sphorb

/home/imonalc/anaconda3/envs/keypoints/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def sort_key(pts1, pts2, desc1, desc2, points):

    ind1 = np.argsort(pts1[:,2].numpy(),axis = 0)[::-1]
    ind2 = np.argsort(pts2[:,2].numpy(),axis = 0)[::-1]

    max1 = np.min([points,ind1.shape[0]])
    max2 = np.min([points,ind2.shape[0]])

    ind1 = ind1[:max1]
    ind2 = ind2[:max2]

    pts1 = pts1[ind1.copy(),:]
    pts2 = pts2[ind2.copy(),:]

    desc1 = desc1[:,ind1.copy()]
    desc2 = desc2[:,ind2.copy()]

    pts1 = np.concatenate((pts1[:,:2], np.ones((pts1.shape[0],1))), axis = 1 )
    pts2 = np.concatenate((pts2[:,:2], np.ones((pts2.shape[0],1))), axis = 1 )

    desc1 = np.transpose(desc1,[1,0]).numpy()
    desc2 = np.transpose(desc2,[1,0]).numpy()

    return pts1, pts2, desc1, desc2


def mnn_mather(desc1, desc2, method="mean_std"):
    sim = desc1 @ desc2.transpose()
    if method == "mean_std":
        k = 4
        threshold = sim.mean() + k * sim.std()
    sim_raw = sim.copy()
    sim[sim < threshold] = 0
    nn12 = np.argmax(sim, axis=1)
    nn21 = np.argmax(sim, axis=0)
    ids1 = np.arange(0, sim.shape[0])
    mask = (ids1 == nn21[nn12])
    matches = np.stack([ids1[mask], nn12[mask]])
    return matches.transpose(), sim_raw


def matched_points(pts1, pts2, desc1, desc2, opt, args_opt, match='ratio', use_new_method=0):
    if opt[-1] == 'p':
        porce = int(opt[:-1])
        n_key = int(porce/100 * pts1.shape[0])
    else:
        n_key = int(opt)

    s_pts1  = pts1.copy()[:n_key,:]
    s_pts2  = pts2.copy()[:n_key,:]
    s_desc1 = desc1.copy().astype('float32')[:n_key,:]
    s_desc2 = desc2.copy().astype('float32')[:n_key,:]

    if 'orb' in args_opt:
        s_desc1 = s_desc1.astype(np.uint8)
        s_desc2 = s_desc2.astype(np.uint8)
        hamming_distances = np.array([[cv2.norm(d1, d2, cv2.NORM_HAMMING) for d2 in s_desc2] for d1 in s_desc1])
        sim = -hamming_distances
        nn12 = np.argmin(hamming_distances, axis=1)
        nn21 = np.argmin(hamming_distances, axis=0)
        ids1 = np.arange(len(s_desc1))
        mask = (ids1 == nn21[nn12])
        matches = [cv2.DMatch(i, j, 0) for i, j in zip(ids1[mask], nn12[mask])]
    else:
        matches_idx, sim = mnn_mather(s_desc1, s_desc2)
        matches = [cv2.DMatch(i, j, 0) for i, j in matches_idx]

    M = np.zeros((2,len(matches)))
    for ind, match in zip(np.arange(len(matches)),matches):
        M[0,ind] = match.queryIdx
        M[1,ind] = match.trainIdx


    return s_pts1, s_pts2, s_pts1[M[0,:].astype(int),:3], s_pts2[M[1,:].astype(int),:3], sim


def get_error(x1, x2, Rx, Tx):

    S = computeEssentialMatrixByRANSAC(x1, x2)
    I = S[1]
    I = I.astype(np.int64)

    x1 = x1[I,:]
    x2 = x2[I,:]

    F = calc_ematrix(x1,x2)


    R1,R2,T1,T2 = decomposeE(F)

    R,T = choose_rt(R1,R2,T1,T2,x1,x2)

    R_error = r_error(Rx,R)
    T_error = t_error(Tx,T)

    return R_error, T_error


def get_descriptor(descriptor):
    if descriptor == 'sphorb':
        return 'sphorb', 'erp', 640, 0
    elif descriptor == 'sift':
        return 'sift', 'erp', 512, 0
    elif descriptor == 'tsift':
        return 'sift', 'tangent', 512, 0
    elif descriptor == 'orb':
        return 'orb', 'erp', 512, 0
    elif descriptor == 'torb':
        return 'orb', 'tangent', 512, 0
    elif descriptor == 'spoint':
        return 'superpoint', 'erp', 512, 0
    elif descriptor == 'tspoint':
        return 'superpoint', 'tangent', 512, 0
    elif descriptor == 'alike':
        return 'alike', 'erp', 512, 0
    elif descriptor == 'talike':
        return 'alike', 'tangent', 512, 0
    elif descriptor == 'Proposed':
        return 'superpoint', 'tangent', 512, 1
    elif descriptor == 'Ltspoint':
        return 'superpoint', 'tangent', 512, 2
    elif descriptor == 'Ftspoint':
        return 'superpoint', 'tangent', 512, 3


def get_error(x1, x2, Rx, Tx):

    S = computeEssentialMatrixByRANSAC(x1, x2)
    I = S[1]
    I = I.astype(np.int64)

    x1 = x1[I,:]
    x2 = x2[I,:]

    F = calc_ematrix(x1,x2)


    R1,R2,T1,T2 = decomposeE(F)

    R,T = choose_rt(R1,R2,T1,T2,x1,x2)

    R_error = r_error(Rx,R)
    T_error = t_error(Tx,T)

    return R_error, T_error


def AUC(ROT, TRA, MET, L):

    RAUC  = np.zeros(len(L))
    TAUC  = np.zeros(len(L))

    for index, t in enumerate(L):
        ids = np.where(ROT<np.radians(t))[0]
        RAUC[index] = len(ids)/len(ROT)

    for index, t in enumerate(L):
        ids = np.where(TRA<np.radians(t))[0]
        TAUC[index] = len(ids)/len(TRA)

    return RAUC, TAUC, np.array(MET)


def get_data(DATAS):
    if len(DATAS) == 1:
        data = DATAS[0]
    elif set(['Urban1','Urban2','Urban3','Urban4']) == set(DATAS):
        data = 'Outdoor'
    elif set(['Realistic','Interior1','Interior2','Room','Classroom']) == set(DATAS):
        data = 'Indoor'
    elif set(['Urban1_R','Urban2_R','Urban3_R','Urban4_R','Realistic_R','Interior1_R','Interior2_R','Room_R','Classroom_R']) == set(DATAS):
        data = 'OnlyRot'
    elif set(['Urban1_T','Urban2_T','Urban3_T','Urban4_T','Realistic_T','Interior1_T','Interior2_T','Room_T','Classroom_T']) == set(DATAS):
        data = 'OnlyTra'
    else:
        data = ''
        for DA in DATAS:
            data+=DA

    return data


def get_kd(array):

    array = np.array(array)
    delimiter = int(array[-1])
    A = array[:-1]
    K = A[:delimiter].reshape(-1,3)
    D = A[delimiter:].reshape(-1,32)
    return K,D


def normalize_features(features):
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms

    threshold = 0.2
    normalized_features = np.minimum(normalized_features, threshold)

    norms = np.linalg.norm(normalized_features, axis=1, keepdims=True)
    normalized_features /= norms

    return normalized_features


In [3]:
points = 500
match = "ratio"
solver = "None"
inliers = "5PA"
descriptor = "tsift"
path = "./data/Farm_pair"  #"./data/data_100/Room/0"

In [24]:
def test(points = 500,
match = "ratio",
descriptor = "tsift",
path = "./data/Farm_pair",):
    t0 = time.time()
    descriptor = descriptor

    opt, mode, sphered, use_our_method = get_descriptor(descriptor)
    base_order = 0  # Base sphere resolution
    sample_order = 8  # Determines sample resolution (10 = 2048 x 4096)
    scale_factor = 1.0  # How much to scale input equirectangular image by
    save_ply = False  # Whether to save the PLY visualizations too
    dim = np.array([2*sphered, sphered])

    path_o = path + '/O.png'
    path_r = path + '/R.png'
    img_o = load_torch_img(path_o)[:3, ...].float()
    img_o = F.interpolate(img_o.unsqueeze(0), scale_factor=scale_factor, mode='bilinear', align_corners=False, recompute_scale_factor=True).squeeze(0)
    img_r = load_torch_img(path_r)[:3, ...].float()
    img_r = F.interpolate(img_r.unsqueeze(0), scale_factor=scale_factor, mode='bilinear', align_corners=False, recompute_scale_factor=True).squeeze(0)
    img_o = torch2numpy(img_o.byte())
    img_r = torch2numpy(img_r.byte())
    img_o = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
    img_r = cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB)
    height_threshold = 0.7 * img_o.shape[0]


    print(path_o)
    t1 = time.time()
    print("image:", t1-t0)
    if opt != 'sphorb':
        corners = tangent_image_corners(base_order, sample_order)
        pts1, desc1 = process_image_to_keypoints(path_o, corners, scale_factor, base_order, sample_order, opt, mode)
        print(descriptor, len(pts1))
        pts2, desc2 = process_image_to_keypoints(path_r, corners, scale_factor, base_order, sample_order, opt, mode)
        pts1[pts1[:,0] > img_o.shape[1], 0] -= img_o.shape[1]
        pts2[pts2[:,0] > img_o.shape[1], 0] -= img_o.shape[1]
    else:      
        os.chdir('SPHORB-master/')
        pts1, desc1 = get_kd(sphorb.sphorb(path_o, points))
        pts2, desc2 = get_kd(sphorb.sphorb(path_r, points))
        os.chdir('../')
    if opt == "sift":
        desc1 = normalize_features(desc1)
        desc2 = normalize_features(desc2)

    #print(desc1.shape, desc1)
    pts1, pts2, desc1, desc2 = sort_key(pts1, pts2, desc1, desc2, points)
    t2 = time.time()
    print("detection:", t2-t1)
    if len(pts1.shape) == 1:
        pts1 = pts1.reshape(1,-1)

    print(desc1.shape)
    s_pts1, s_pts2, x1, x2, sim = matched_points(pts1, pts2, desc1, desc2, "100p", opt, match, use_new_method=use_our_method)
    print()
    #print(x1[:5])
    #print(desc1[:5],)
    sim = -sim
    sim = (sim - np.min(sim)) / (np.max(sim) - np.min(sim))
    #print(sim[:5])
    return sim, x1, x2


In [25]:
sim_p, x1_p, x2_p = test(descriptor="Proposed")
sim_sp, x1_sp, x2_sp = test(descriptor="spoint")

./data/Farm_pair/O.png
image: 0.30002284049987793
Proposed 606
detection: 1.2272167205810547
(500, 256)

[[0.7387523  0.57731444 0.75546753 ... 0.7078584  0.6948713  0.81339633]
 [0.74364936 0.688245   0.75579    ... 0.70178986 0.7390656  0.7181975 ]
 [0.32185107 0.71577114 0.73720515 ... 0.7277505  0.7387182  0.691754  ]
 [0.3534079  0.75297564 0.7653046  ... 0.7411455  0.6986666  0.6885239 ]
 [0.55149937 0.7949119  0.75320345 ... 0.7720933  0.83194184 0.7562096 ]]
./data/Farm_pair/O.png
image: 0.320812463760376
spoint 21781
detection: 1.4440572261810303
(500, 256)

[[0.6358735  0.6052344  0.6737824  ... 0.66104    0.67065793 0.68292046]
 [0.6872951  0.64161325 0.70150363 ... 0.34419656 0.6179201  0.68636084]
 [0.446369   0.6168526  0.5468437  ... 0.788171   0.7226232  0.8085861 ]
 [0.794013   0.77078307 0.54308    ... 0.81109136 0.6840271  0.6667871 ]
 [0.43693468 0.7499981  0.543808   ... 0.7931081  0.6592515  0.70623964]]


In [26]:
sim_list = [sim_sp, sim_p]
nm_list = ["SPoint", "Tspoint"]

In [27]:
sim_dict = {
    "spoint": (sim_sp, x1_sp, x2_sp),
    "tspoint":( sim_p, x1_p, x2_p),
}

In [28]:
import pickle
with open("sim_matching.pickle", mode='wb') as f:
    pickle.dump(sim_dict,f)

In [57]:
threshold = np.percentile(sim_p, 95)
print(threshold)

0.8222279936075211


In [47]:
simp_idx_list = []
simsp_idx_list = []
for idx_x1 in range(500):
    for idx_x2 in range(500):
        simp_idx_list.append([sim_p[idx_x1][idx_x2], idx_x1, idx_x2])
        simsp_idx_list.append([sim_sp[idx_x1][idx_x2], idx_x1, idx_x2])


simp_idx_list.sort(reverse=True) 
simsp_idx_list.sort(reverse=True) 

In [59]:
simp_idx_x1_list = []
simp_idx_x2_list = []
simsp_idx_x2_list = []
simsp_idx_x1_list = []

simp_idx_list_unique = []
for idx in range(len(simp_idx_list)):
    sim, idx_x1, idx_x2 = simp_idx_list[idx]
    flag = False
    if idx_x1 in simp_idx_x1_list: flag = True
    if idx_x2 in simp_idx_x2_list: flag = True
    simp_idx_x1_list.append(idx_x1)
    simp_idx_x2_list.append(idx_x2)
    if flag: continue
    simp_idx_list_unique.append(sim)

print(len(simp_idx_list_unique))

simsp_idx_list_unique = []

143


In [60]:
simp_idx_list_unique

[1.0,
 0.97981495,
 0.97968006,
 0.97908735,
 0.9772731,
 0.97233063,
 0.9707402,
 0.96797967,
 0.9676867,
 0.9657811,
 0.96300775,
 0.96127665,
 0.9588784,
 0.9560634,
 0.9550731,
 0.953602,
 0.95334524,
 0.9523523,
 0.95204896,
 0.95025635,
 0.94958466,
 0.94876516,
 0.94719046,
 0.9465559,
 0.94649017,
 0.9438842,
 0.9426292,
 0.94251347,
 0.9416456,
 0.94115883,
 0.9410301,
 0.94048613,
 0.94012636,
 0.9389956,
 0.93896884,
 0.9389643,
 0.9387997,
 0.9377583,
 0.9377183,
 0.9372677,
 0.9371908,
 0.9360279,
 0.9359062,
 0.93582666,
 0.93522376,
 0.93496346,
 0.9339908,
 0.93377215,
 0.93345255,
 0.9328581,
 0.9321327,
 0.932047,
 0.9315888,
 0.9304759,
 0.9293803,
 0.9292498,
 0.9291499,
 0.92849505,
 0.927503,
 0.9273187,
 0.92712766,
 0.9268669,
 0.9266633,
 0.92542404,
 0.9251957,
 0.92477816,
 0.9237096,
 0.92365277,
 0.92292607,
 0.922713,
 0.9222999,
 0.9213546,
 0.92119235,
 0.92059153,
 0.92056876,
 0.9204355,
 0.9190096,
 0.9188801,
 0.91813797,
 0.9180027,
 0.91732913,
 0.

In [49]:
len(simp_idx_list)

250000

In [9]:
import scipy